# Chicago Traffic Incidents Involving Cyclists/Pedestrians in Last 12 Months
### Prepared by: Michael Goodman
>### Data Source: Chicago Data Portal
>#### Vehicles API Documentation: https://dev.socrata.com/foundry/data.cityofchicago.org/68nd-jvt3
>#### People API Documentation: https://dev.socrata.com/foundry/data.cityofchicago.org/u6pd-qa9d

### Libraries to Import

In [378]:
#Imports
#use pip to install any needed libraries below
import pandas as pd
import requests
import json
import time
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from sodapy import Socrata
from cdp_secrets import *
print('completed at ' + str(datetime.now()))

completed at 2023-01-05 20:56:54.399284


### Chicago Data Portal Connection

In [379]:
#Constants
chicago_url="data.cityofchicago.org"
vehicle_api_root="68nd-jvt3"
people_api_root="u6pd-qa9d"
crash_api_root="85ca-t3if"
district_root="z8bn-74gv"
beat_root="n9it-hstw"

#Chicago Data Portal Connection
cdp = Socrata(chicago_url,
                 token,
                 username,
                 password)
print('completed at ' + str(datetime.now()))

completed at 2023-01-05 20:56:58.586951


### Function to call API and format to DataFrame

In [380]:
def callAPI(root: str, filter: str) -> pd.DataFrame:
    results=cdp.get(root, query=filter)
    df=pd.DataFrame.from_records(results)
    return df
print('completed at ' + str(datetime.now()))

completed at 2023-01-05 20:57:02.064308


### Create string variables for today and one year ago

In [381]:
#Get dates for the last 12 months
today = datetime.now().strftime("%Y-%m-%d")

year_ago = (datetime.now() - relativedelta(years=1)).strftime("%Y-%m-%d")
print(today)
print(year_ago)
print('completed at ' + str(datetime.now()))

2023-01-05
2022-01-05
completed at 2023-01-05 20:57:04.324759


### Queries for API Calls. Filters based on project specification

In [404]:
#limit required for SoQL query, as we are filtering for one year limit is set high to capture all records
limit='1000000000'

#Filters use SQL structure, some where clauses use a different format. Refer to API Documentation for correct syntax
vehicle_filter= """Select crash_record_id,crash_date,make,model,vehicle_type 
                    where vehicle_type is not null and crash_date between '%s' and '%s'
                    limit %s"""%(year_ago,today,limit) 

people_filter= """Select person_id,crash_record_id,crash_date,person_type,age,sex,
                injury_classification,pedpedal_action,pedpedal_visibility,pedpedal_location
                Where CRASH_DATE between '%s' and '%s' and
                (person_type='PEDESTRIAN' or person_type='BICYCLE') limit %s"""% (year_ago,today,limit)

crash_filter="""Select crash_record_id,crash_date,beat_of_occurrence as beat_num,crash_month,crash_day_of_week,location
                where crash_date between '%s' and '%s' limit %s"""% (year_ago,today,limit)

district_filter="""Select distinct district,district_name limit 1000"""

beat_filter="""Select beat_num,district limit 300"""

print('completed at ' + str(datetime.now()))

completed at 2023-01-05 21:19:20.058851


### Create DataFrames from API Results

In [383]:
#People Table
people_df=callAPI(people_api_root,people_filter)

print('completed at ' + str(datetime.now()))

completed at 2023-01-05 20:57:11.596833


In [384]:
#Vehicle Table
vehicle_df=callAPI(vehicle_api_root,vehicle_filter)

print('completed at ' + str(datetime.now()))

completed at 2023-01-05 20:57:19.540316


In [406]:
#Crash Table
crash_df=callAPI(crash_api_root,crash_filter)

print('completed at ' + str(datetime.now()))

completed at 2023-01-05 21:19:58.003449


In [415]:
#Beat To District Key Table

#Police Beat
beat_df=callAPI(beat_root,beat_filter)

#Remove leading 0 from district field
beat_df['district']=pd.to_numeric(beat_df['district'])
beat_df['district']=beat_df['district'].apply(lambda x: str(x))

#Remove leading 0 from beat_num field
beat_df['beat_num']=pd.to_numeric(beat_df['beat_num'])
beat_df['beat_num']=beat_df['beat_num'].apply(lambda x: str(x))

#District table
district_df=callAPI(district_root,district_filter)

#join together to create matching table
district_key=pd.merge(beat_df,district_df,on='district',how='left')

print(district_key)
print('completed at ' + str(datetime.now()))

    beat_num district   district_name
0       1713       17     Albany Park
1       3100       31             NaN
2       1651       16  Jefferson Park
3       1914       19       Town Hall
4       1915       19       Town Hall
..       ...      ...             ...
272      314        3  Grand Crossing
273      825        8    Chicago Lawn
274      313        3  Grand Crossing
275      823        8    Chicago Lawn
276      312        3  Grand Crossing

[277 rows x 3 columns]
completed at 2023-01-05 21:28:30.851114


### Join DataFrames to create project Data Model

In [416]:
#Join People and Vehicle data sets on crash_record_id
people_vehicle=pd.merge(people_df,vehicle_df[['crash_record_id','make','model','vehicle_type']], on='crash_record_id', how='left')
#print(people_vehicle)
print('completed at ' + str(datetime.now()))

completed at 2023-01-05 21:28:38.212126


In [417]:
#Join prior dataset with crash information
crash_pv=pd.merge(people_vehicle,crash_df, on='crash_record_id', how='left')
#print(crash_pv)
print('completed at ' + str(datetime.now()))

completed at 2023-01-05 21:28:40.032027


In [418]:
#Join prior dataset with district ID and Name
merge_df=pd.merge(crash_pv,district_key, on='beat_num', how='left')
#Declare final data model
model=merge_df
print(model)

print('completed at ' + str(datetime.now()))

     person_id                                    crash_record_id  \
0     O1493509  1f0cafca3ea46ae86def5392688733931c7c8157bb276e...   
1     O1493466  05463f20a4cb41d431ca63c785170683919a6c24043473...   
2     O1493480  d70115d85901528502c8e3760835549b9fd48e7a438863...   
3     O1493407  d8987c29a77244ac4a4ecaa2b041f627eab86d2243232a...   
4     O1493351  58c7e9c0cba138b804a300dbbdea54437bccef582f08eb...   
...        ...                                                ...   
4671  O1258162  38bbc2843a7e6220695848659de6999284479481708a4e...   
4672  O1259041  49edea243c3b8a868504f514e4627089ac1335b77b1be3...   
4673  O1259041  49edea243c3b8a868504f514e4627089ac1335b77b1be3...   
4674  O1257916  7b513e0b02b824ba9b92db19e5a7ddb60949b22444e088...   
4675  O1260978  b6071c55be86f528b8169b3cc2d27855158c7077d93242...   

                 crash_date_x person_type  age sex     injury_classification  \
0     2023-01-04T19:15:00.000  PEDESTRIAN   49   M                     FATAL   
1     2023-

### Print to Excel

In [419]:
model.to_excel('traffic_study_model_'+today+'.xlsx',index=True)
print('completed at '+str(datetime.now()))

completed at 2023-01-05 21:28:54.724036
